In [75]:
import plotly.express as px
import plotly.graph_objects as go

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

import pandas as pd
import numpy as np

In [76]:
df = pd.read_csv('https://raw.githubusercontent.com/artist0123/ProjectPSIT/main/master.csv', engine = 'python', sep = ',')

In [77]:
df_country = df.groupby(['Country', 'year']).sum().reset_index()
df_country.replace(0, 1, inplace=True)

In [78]:
code_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df_country_code = df_country.merge(code_df, left_on='Country', right_on='COUNTRY', how='left')
df_year = df.groupby('year').sum()
df_chocolate = df.groupby(['sex', 'age'], as_index=False).sum()

In [81]:
 # Build App
YEARS = [x for x in range(1987, 2015)]
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

card1 = [dbc.CardHeader(children=[
                dcc.Graph(id='world-choropleth'),
                        ]),#CardHeader
                        dbc.CardBody(children=[
                            dcc.Slider(
                            id='year-slider',
                            min=min(YEARS),
                            max=max(YEARS),
                            value=min(YEARS),
                            marks={i:"%d" % i for i in YEARS},
                            )
                        ]),#CardBody
                    ]

card2 = [dbc.CardHeader(children=[
            dcc.Graph(id='everest-graph'),
            ]),#CardHeader
            dbc.CardBody([
                html.H5("Card title", className="card-title"),
                html.P(
                    "This is some card content that we'll reuse",
                    className="card-text",
                )
            ])
        ]

card3 = [dbc.CardHeader(children=[
                dcc.Graph(id='chocolatebar-graph'),
                        ]),#CardHeader
                    ]

app.layout = html.Div(children=[
            html.Div(
                id="header",
                className='jumbotron',
                children=[
                    html.H1(children="Data Visualization Dashboard")
                ]),
            dbc.Container(fluid=False,
            children=[#Container
                dbc.Row([#Row
                    dbc.Col([#Col1
                        dbc.Card(card1, outline=True),
                        dbc.Card(card2, outline=True),
                        dbc.Card(card3, outline=True),
                    ])#Col1
            ])#Row
        ])#Container
    ])#Div1

In [73]:
# Define callback to update graph
@app.callback(
    Output(component_id='world-choropleth', component_property='figure'),
    Output(component_id='everest-graph', component_property='figure'),
    Output(component_id='chocolatebar-graph', component_property='figure'),
    Input(component_id='year-slider', component_property='value')
)
def update_figure(year):
    df_choropleth = df_country_code.query('year=='+str(year))
    world_suicide = go.Figure(data=go.Choropleth(
    locations = df_choropleth['CODE'],
    z = df_choropleth['suicides_no'],
    text = df_choropleth['Country'],
    colorscale = 'Oryel',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.2,
    colorbar_title = 'Suicide Cases',
    ))

    world_suicide.update_layout(
    autosize=True,
    title_text='Global Suicide Cases in ' + str(year),
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
        )
    )
    everest_graph = px.line(df_year, x=df_year.index, y="suicides_no", title='Worldwide Suicide case (1985 - 2015)')

    chocolatebar_graph = px.bar(df_chocolate, x='sex', y="suicides_no", color='age', barmode='group', title='Worldwide Suicide by Age and Gender (1985 - 2015)')

    return world_suicide, everest_graph, chocolatebar_graph

In [74]:
# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
